# Talbot Carpet Raw Loader (B4)
Downloads the Zenodo archive `10.5281/zenodo.14002229`, extracts the HDF5/TIFF files,
re-averages the intensity stack, and reproduces the FFT z-slice with ±1⁄14 peaks.

## About the Talbot Raw Dataset
This notebook demonstrates how to download, extract, and process the Talbot carpet raw dataset from Zenodo (DOI: 10.5281/zenodo.14002229). The dataset contains intensity stacks in HDF5 format and TIFF images for optical analysis.

In [ ]:
import requests, zipfile, io, numpy as np, h5py, matplotlib.pyplot as plt
import imageio.v3 as iio

# download Zenodo zip (≈80 MB) to memory
url = "https://zenodo.org/record/14002229/files/talbot_raw.zip?download=1"
raw = io.BytesIO(requests.get(url, timeout=60).content)
zipfile.ZipFile(raw).extractall("/tmp/talbot")

# load intensity H5 stack and average
with h5py.File("/tmp/talbot/fixed_order_intense.h5", "r") as h5:
    stack = h5["intensity"][:]
avg = stack.mean(axis=0)

# FFT z-slice
fft = np.log(np.abs(np.fft.fftshift(np.fft.fft2(avg))))
zslice = fft[fft.shape[0]//2, :]
plt.plot(zslice); plt.title("FFT z-slice"); plt.show()

In [ ]:
# Check extracted files and show available data
import os
for fname in os.listdir("/tmp/talbot"):
    print(fname)

## Visualize a Sample TIFF Image
The dataset also includes TIFF images. Let's display one as an example.

In [ ]:
# Display a sample TIFF image from the dataset
import matplotlib.pyplot as plt
import imageio.v3 as iio

tiff_path = "/tmp/talbot/fixed_order_0000.tif"
img = iio.imread(tiff_path)
plt.imshow(img, cmap="gray")
plt.title("Sample Talbot TIFF Image")
plt.axis("off")
plt.show()

## Notes
- The HDF5 stack is averaged along the z-axis to produce a 2D intensity map.
- The FFT z-slice visualizes spatial frequency content, revealing the ±1⁄14 peaks characteristic of the Talbot effect.
- For more details, see the dataset's README and Zenodo page.

### Troubleshooting LaTeX Unicode Errors
If you encounter LaTeX errors about Unicode characters (e.g., α, Λ, ħ, ≈, −, ℓ), you can:
- Replace Unicode symbols in your .tex files with LaTeX macros (e.g., `\alpha`, `\Lambda`, `\hbar`, `\approx`, `\ell`, `-`).
- Or, use XeLaTeX for Unicode support by running:

    latexmk -xelatex -silent torus_book.tex

This will allow LaTeX to process Unicode characters natively.

### Resolving Git Submodule Path Errors
If you see errors like:

    fatal: No url found for submodule path 'data/interferometer/upstream/GWDetectorZoo' in .gitmodules

it means a submodule is referenced but not defined. To remove the obsolete submodule, run:

    git submodule deinit -f data/interferometer/upstream/GWDetectorZoo
    git rm -f data/interferometer/upstream/GWDetectorZoo
    git commit -m "Remove obsolete submodule GWDetectorZoo"
    git push

If you need the submodule, add the correct entry to `.gitmodules` and initialize it with:

    git submodule update --init --recursive

### Additional Notes
- The script `scripts/audit_and_regen_docx.sh` exists but is currently empty. If needed, add your audit and regeneration logic to this file and ensure it is executable.
- The script `scripts/convert_docx_to_tex.sh` is present and functional for DOCX to LaTeX conversion.
- If you encounter further workflow or build issues, check the log files (e.g., `torus_book.log`) for details.